# Piping a RunnableParallel with Other Runnables

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [2]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [3]:
chat = ChatOllama(model = 'llama3.2',
                  temperature = 0,
                  num_predict=100)

In [4]:
string_parser = StrOutputParser()

In [5]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [6]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [7]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Automate the Boring Stuff with Python" by Al Sweigart\n2. "Python Crash Course" by Eric Matthes\n3. "Learning Python" by Mark Lutz',
 'projects': '1. Building a Web Scraper using BeautifulSoup and Requests\n2. Creating a Chatbot with Natural Language Processing (NLP) using NLTK and Keras\n3. Developing a Game of Tic-Tac-Toe using Minimax Algorithm and GUI with Tkinter or PyQt'}

In [8]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | chat 
              | string_parser
             )

In [9]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

In [10]:
print(chain_time2.invoke({'programming language':'Python'}))

Based on your intermediate level programming skills, I'll provide an estimate of the time required to complete the literature and projects.

**Literature:**

1. "Automate the Boring Stuff with Python" by Al Sweigart:
	* This book is designed for beginners, so it might not be as challenging for you.
	* Estimated completion time: 2-4 weeks (10-20 hours)
2. "Python Crash Course" by Eric Matthes:
	* This book covers


In [11]:
chain_time2.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOllama |                      | ChatOllama |     
    +------------+                      +------------+     
           *                                   *           
           *                            